# Visualizing the performance of deep learning models with `Tensorboard`

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**There are various tools for measuring the performance of a deep learning model, like [Neptune AI](https://www.google.com/aclk?sa=l&ai=DChcSEwj1xIKopMn5AhVB5VwKHYIcAVwYABAAGgJjZQ&sig=AOD64_3vHRiEXqPZnFvdKHZez_oorbqA6w&q&adurl&ved=2ahUKEwjv6PynpMn5AhXZgpUCHR1CCXUQ0Qx6BAgCEAE) and [Weights and Biases](https://wandb.ai/site). In this notebook, we’ll teach how to use _TensorFlow’s_ open-source _visualization toolkit: [TensorBoard](https://www.tensorflow.org/tensorboard)_.**

![tensordboard](https://www.tensorflow.org/tensorboard/images/tensorboard.gif)


### If you don't have tensorboard installed, you can quickly get this package by:

- `pip install tensorboard` in your terminal/CLI;
- `%pip install tensorboard` in a code cell of your jupyter notebook;
- `!pip install tensorboard` in a code cell of your colab notebook.

_Now, you just need to load the tensorboard into your Notebook and set a log directory (this directory is where tensorboard will store all the logs from your ML experiments._


In [1]:
%load_ext tensorboard
log_folder = 'logs'


_If you want to reload the tensorboard from before:_

- `%reload_ext tensorboard`

_If you want to clear the current logs to write new ones:_

- `rm -rf logs`

_If you want to add a timestamp to your logs:_

- `log_folder = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")`


**Let's use the IMDB sentiment-analysis dataset for the sake of this explanation.**

![IMDB](https://upload.wikimedia.org/wikipedia/commons/6/69/IMDB_Logo_2016.svg)


In [2]:
import tensorflow as tf
from keras import layers
from keras.datasets import imdb
from keras_preprocessing.sequence import pad_sequences


vocabulary = 2000 # Number of words to consider as features.
max_len = 500 # Cuts off texts after this number of words (among max_features most common words).

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocabulary)

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

model = tf.keras.models.Sequential()
model.add(layers.Embedding(vocabulary, 128,
                            input_length=max_len,
                            name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 3

Now that you have a model to track, you need to set your tensorboard callback during the model’s fit method (this is the method responsible for logging events such as **activation histograms,** **metrics plots,** **training graph visualizations**.).

Parameters the **TensorBoard callback** takes:

- **log_dir**: where to store the logs;
- **histogram_freq**: the frequency at which to compute activation and weight histograms for layers of the model (_0 means that histograms will not be computed_);
- **write_graph**: `True` will make graph be visualized in TensorBoard;
- **write_images**: `True` will make model weights be visualized as an image in TensorBoard;
- **update_freq**: determines how **losses** and **metrics** are written to TensorBoard (_when set to `epoch` they are written after every epoch_);
- **profile_batch**: determines which batches will be profiled (_by default, the second batch is profiled. Setting profile_batch to 0 disables profiling_);
- **embeddings_freq**: the frequency at which the embedding layers will be visualized (_setting to 0 means that the embeddings will not be visualized_).


In [3]:

callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_folder,
                                           histogram_freq=1,
                                           write_graph=True,
                                           write_images=True,
                                           update_freq='epoch',
                                           profile_batch=2,
                                           embeddings_freq=1)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split= 0.2,
                    callbacks=[callbacks],
                    verbose=1
                    )
test_loss_score, test_acc_score = model.evaluate(x_test, y_test)
print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')

Version:  2.10.0
Eager mode:  True
GPU is available
Epoch 1/20
157/157 [==============================] - 12s 41ms/step - loss: 0.6486 - acc: 0.6292 - val_loss: 0.4802 - val_acc: 0.7848
Epoch 2/20
157/157 [==============================] - 6s 35ms/step - loss: 0.4595 - acc: 0.8449 - val_loss: 0.4489 - val_acc: 0.8566
Epoch 3/20
157/157 [==============================] - 6s 38ms/step - loss: 0.3901 - acc: 0.8737 - val_loss: 0.4749 - val_acc: 0.8576
Epoch 4/20
157/157 [==============================] - 6s 41ms/step - loss: 0.3513 - acc: 0.8941 - val_loss: 0.5078 - val_acc: 0.8698
Epoch 5/20
157/157 [==============================] - 8s 54ms/step - loss: 0.2954 - acc: 0.9158 - val_loss: 0.5811 - val_acc: 0.8692
Epoch 6/20
157/157 [==============================] - 9s 59ms/step - loss: 0.2681 - acc: 0.9270 - val_loss: 0.5944 - val_acc: 0.8656
Epoch 7/20
157/157 [==============================] - 10s 61ms/step - loss: 0.2125 - acc: 0.9495 - val_loss: 0.8601 - val_acc: 0.8446
Epoch 8/20
157/

### Basic TensorBoard Guide

- **The Scalars tab shows changes in the loss and metrics over the epochs. It can be used to track other scalar values such as learning rate and training speed.**
- **The images tab shows the weights. Adjusting the slider displays the weights at various epochs.**
- **The graph tab shows your model’s layers. You can use this to check if the architecture of the model looks as intended.**
- **The distribution tab shows the distribution of tensors (_the distribution of the weights and biases over each epoch_)**.
- **The histograms tab show the distribution of tensors over time.**

**If you installed TensorBoard via pip, you can launch it via the command line:**

> `tensorboard --logdir=logs`

**On a Notebook, you can launch it using:**


In [8]:
from tensorboard import notebook

notebook.list() 
notebook.display(port=6006, height=1000)


Known TensorBoard instances:
  - port 6006: logdir logs (started 0:01:46 ago; pid 14840)
  - port 6006: logdir logs (started 0:03:43 ago; pid 15496)
  - port 6006: logdir logs (started 0:01:26 ago; pid 23124)
Selecting TensorBoard with logdir logs (started 0:01:26 ago; port 6006, pid 23124).


---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).
